### PRODUCT SEARCH IN ECOMMERCE

In [5]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input

def load_and_preprocess_images(image_dir, target_size=(224, 224)):
    image_data = []
    image_paths = []

    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, target_size)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = preprocess_input(img)  # ResNet50 preprocessing
                image_data.append(img)
                image_paths.append(img_path)
    
    return np.array(image_data), image_paths


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tqdm import tqdm

# Load pretrained ResNet50 (exclude final classification layer)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)
# Save the model

def extract_features(images, model):
    features = []
    for img in tqdm(images):
        img_batch = np.expand_dims(img, axis=0)
        feature = model.predict(img_batch, verbose=0)
        features.append(feature.flatten())
    return np.array(features)


In [7]:
# Load and preprocess
image_dir = '../data/raw'  # adjust this to your folder
images, image_paths = load_and_preprocess_images(image_dir)

# Extract features
features = extract_features(images, model)

# Save features and image paths
np.save('../features/resnet50_features.npy', features)

import pickle
with open('../features/image_paths.pkl', 'wb') as f:
    pickle.dump(image_paths, f)

# Save the model
model.save('../models/resnet50_model.h5')
print("Features and paths saved.")


100%|██████████| 118/118 [01:35<00:00,  1.23it/s]


Features and paths saved.
